In [1]:
# importing libraries
import pandas as pd
import numpy as np

from surprise import Dataset, Reader
from surprise import SVD, NMF, KNNWithZScore, CoClustering
from surprise.model_selection import ShuffleSplit
from surprise import accuracy

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler

from collections import defaultdict
from statistics import mean
import random

In [2]:
# Train Triplets Data
train_triplets = 'K:/Datasets/EchoNest/Train Triplets/train_triplets.txt'
train_ratings = pd.read_csv(train_triplets, sep="\t", header=None)
train_ratings.columns = ['user_id', 'song_id', 'listen_count']

# Spotify Metadata 
song_metadata = pd.read_csv('K:/Notebook Files/spotify_metadata.csv')

# get ratings for songs that are in the metadata
ratings = train_ratings[train_ratings['song_id'].isin(song_metadata['en_song_id'])]

In [3]:
# how many songs each user has listened to
user_counts = ratings.groupby('user_id')['song_id'].count()
user_counts.describe().apply(lambda x: format(x, 'f'))

count    1019302.000000
mean          45.680929
std           55.784434
min            1.000000
25%           15.000000
50%           26.000000
75%           53.000000
max         4228.000000
Name: song_id, dtype: object

In [4]:
# how many users listen to the same song on average
song_user = ratings.groupby('song_id')['user_id'].count()
song_user.describe()

count    354962.000000
mean        131.176470
std         801.194337
min           1.000000
25%           4.000000
50%          14.000000
75%          55.000000
max       90476.000000
Name: user_id, dtype: float64

In [5]:
# filter out users that have listened to less than 16 songs
flt_users = user_counts[user_counts > 15].index.to_list() #25%

# filter out songs that have less than 200 users
flt_songs = song_user[song_user > 14].index.to_list() #50%

# filter out dataset with user and song id's
ratings_flt = ratings[(ratings['user_id'].isin(flt_users)) & (ratings['song_id'].isin(flt_songs))].reset_index(drop=True)

In [16]:
# top 10000 songs with most ratings
top_songs = song_user.sort_values(ascending=False)[:10000].index.to_list()
# filter for only top 10000 popular songs
ratings_flt_pop = ratings_flt[(ratings_flt['song_id'].isin(top_songs))].reset_index(drop=True)

# top 10000 users with most ratings
most_ratings_user = user_counts.sort_values(ascending=False)[:10000].index.to_list()
# filter for only top 10000 users
ratings_flt_pop = ratings_flt_pop[(ratings_flt_pop['user_id'].isin(most_ratings_user))].reset_index(drop=True)

In [17]:
# binning technique 
bins = [0,1,2,3,4,5,6,7,8,9,ratings_flt_pop['listen_count'].max()]
ratings_flt_pop['rating'] = pd.cut(ratings_flt_pop['listen_count'], bins=bins, labels=[1,2,3,4,5,6,7,8,9,10])
ratings_flt_pop['rating'] = ratings_flt_pop.rating.astype('int')

In [18]:
# Initialize Reader class with rating scale from 1 to 10
reader = Reader(rating_scale=(1, 10))

# load dataset class with ratings 
data = Dataset.load_from_df(ratings_flt_pop[['user_id', 'song_id', 'rating']], reader)

In [10]:
# *** Content Based Recommendations ***

top_song_features = song_metadata[(song_metadata['en_song_id'].isin(top_songs))].reset_index(drop=True)

def feature_rec(data, song_id):
    # get data for specific song
    song_artist_data = data[data['en_song_id'] == song_id]
    
    song = song_artist_data['title'].values[0] # song title
    artist = song_artist_data['artist'].values[0] # songs artist
    
    # find songs with similar audio features
    similar_songs = data.copy()
    sound_properties = similar_songs.loc[:, data.columns[6:18].to_list()]
    similar_songs['similarity'] = cosine_similarity(sound_properties, sound_properties.to_numpy()[song_artist_data.index[0], None]).squeeze()
    similar_songs = similar_songs.sort_values(by='similarity', ascending=False)
    similar_songs = similar_songs['en_song_id']

    return similar_songs.iloc[1:2].to_list()

def cb_recommendations(data, user_id):
    # get id's for users most listened song
    song_count = ratings_flt_pop[ratings_flt_pop['user_id'] == user_id]['song_id'].shape[0]
    user_songs = ratings_flt_pop[ratings_flt_pop['user_id'] == user_id].sort_values(by='listen_count', ascending=False)['song_id'][:10]

    # append similar song to list
    recommendations = []
    for song in user_songs:
        sim_songs = feature_rec(data, song)
        for song in sim_songs:
            recommendations.append(song)
    return random.sample(recommendations, 10)

In [11]:
# code from suprise library website

# Return precision and recall at k metrics for each user
def hybrid_recommendations(user_id, predictions, n=20):
    
    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        if(uid == user_id):
            user_est_true[uid].append((round(est, 2), round(true_r, 3), iid))    
    
    cb_list = cb_recommendations(top_song_features, user_id)
    cb_pred = []
    for song_id in cb_list:
        cb_pred.append(algo.predict(user_id, song_id, r_ui=3))
    
    i = 0
    for tpl in list(user_est_true.values())[0]:
        i+=1
        if(tpl[2] in (cb_list)):
            del user_est_true[user_id][i-1]
    
    for uid, iid, true_r, est, _ in cb_pred:
        # print((round(est, 2), round(true_r, 2)))
        user_est_true[uid].insert(random.randint(0, 20), (round(est, 2), round(true_r, 2), iid))    
        # user_est_true[uid].append((round(est, 2), round(true_r, 2), iid))  
    
    user_ratings = []
    recommendations = []
    for est, true_r, song_id in sorted(list(user_est_true.values())[0], key=lambda x: x[0], reverse=True):
        user_ratings.append((est, true_r))
        recommendations.append(song_id)
    
    return user_ratings, recommendations[:n]

In [12]:
# code from suprise library website
# source: https://surprise.readthedocs.io/en/stable/FAQ.html

def precision_recall_at_k_hybrid(user_ratings, k=15, threshold=4):
    
    for est, true_r in user_ratings:
        # How many relevant items
        num_rel = sum((true_r >= threshold) for (est, true_r) in user_ratings)
        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, true_r) in user_ratings[:k])
        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])
        
        # Precision@K: Proportion of recommended items that are relevant
        precision = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1
        # Recall@K: Proportion of relevant items that are recommended
        recall = n_rel_and_rec_k / num_rel if num_rel != 0 else 1
        
    return round(precision, 2), round(recall, 2)

In [13]:
# code from suprise library website
# source: https://surprise.readthedocs.io/en/stable/FAQ.html

# Return precision and recall at k metrics for each user
def precision_recall_at_k(predictions, k=15, threshold=4):
    
    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))
    
    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():
        
        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)
        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)
        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold)) for (est, true_r) in user_ratings[:k])
        
        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1
        
        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1
    
    return precisions, recalls

In [19]:
# algos = [SVD(), NMF(), KNNWithZScore(), CoClustering()]
# test_size = ["0.25", "0.30", "0.40"]

algos = [SVD(), NMF()]
test_size = ["0.30"]
n_splits = 2

user_sample = ratings_flt_pop['user_id'].sample(n=25, random_state=10)

results = {}
hybrid_results = {}

for algorithm in algos:
    algo_dict = {}

    rmse_list = []
    precision_list = []
    recall_list = []
    
    algorithm_name = str(algorithm).split(".")[3].split(" ")[0]
    print(f'\nAlgorithm: {algorithm_name}')
    print('------------------------------')
    
    for size in test_size:
        test_dict = {}
        print(f'\nTest size: {size}')
        print('====================')
        
        kf = ShuffleSplit(n_splits=n_splits, test_size=float(size), shuffle=True, random_state=42)
        rmse_list = []
        precision_list = []
        recall_list = []
        
        for trainset, testset in kf.split(data):
            # training
            print('Model is being trained...')
            algo = algorithm
            algo.fit(trainset) 
            print('Training Successful\n')

            # testing
            print('Model is being tested...')
            predictions = algo.test(testset)
            result = round(accuracy.rmse(predictions, verbose=False), 4)
            print('Testing Successful\n')
            print(f'Testset RMSE is {result}')
            rmse_list.append(result)
            
            # recall and precision @ 20
            precisions, recalls = precision_recall_at_k(predictions, k=15, threshold=4)
            precision = round(sum(prec for prec in precisions.values()) / len(precisions), 4)
            recall = round(sum(rec for rec in recalls.values()) / len(recalls), 4)
            
            print(f'Precision: {precision}')
            print(f'Recall: {recall}')
            print('--------------------')
            
            precision_list.append(precision)
            recall_list.append(recall)
        
        avg_rmse = round(mean(rmse_list), 4)
        avg_precision = round(mean(precision_list), 4)
        avg_recall = round(mean(recall_list), 4)
        
        print('********************')
        print(f'Mean Precision: {avg_precision}')
        print(f'Mean Recall: {avg_recall}')
        
        test_dict['rmse'] = avg_rmse
        test_dict['precision'] = avg_precision
        test_dict['recall'] = avg_recall
        
        algo_dict[f'testset_{size}'] = test_dict
        
    results[algorithm_name] = algo_dict
        
    # Hybrid Testing
    hybrid_metrics = {}

    hybrid_precision = []
    hybrid_recall = []
    for user in user_sample:
        user_ratings, recommendations = hybrid_recommendations(user, predictions)
        precision, recall = precision_recall_at_k_hybrid(user_ratings)
        hybrid_precision.append(precision)
        hybrid_recall.append(recall)
    
    avg_hybrid_precision = np.mean(hybrid_precision)
    avg_hybrid_recall = np.mean(hybrid_recall)
    
    print(avg_hybrid_precision, avg_hybrid_recall)
    
    hybrid_metrics['precision'] = avg_hybrid_precision
    hybrid_metrics['recall'] = avg_hybrid_recall

    hybrid_results[algorithm_name] = hybrid_metrics


Algorithm: SVD
------------------------------

Test size: 0.30
Model is being trained...
Training Successful

Model is being tested...
Testing Successful

Testset RMSE is 2.0946
Precision: 0.7962
Recall: 0.2827
--------------------
Model is being trained...
Training Successful

Model is being tested...
Testing Successful

Testset RMSE is 2.101
Precision: 0.8182
Recall: 0.2802
--------------------
********************
Mean Precision: 0.8072
Mean Recall: 0.2814
0.6936 0.23879999999999998

Algorithm: NMF
------------------------------

Test size: 0.30
Model is being trained...
Training Successful

Model is being tested...
Testing Successful

Testset RMSE is 2.3052
Precision: 0.6306
Recall: 0.3092
--------------------
Model is being trained...
Training Successful

Model is being tested...
Testing Successful

Testset RMSE is 2.3177
Precision: 0.6405
Recall: 0.314
--------------------
********************
Mean Precision: 0.6356
Mean Recall: 0.3116
0.6731999999999999 0.25880000000000003


In [20]:
print(results)

{'SVD': {'testset_0.30': {'rmse': 2.0978, 'precision': 0.8072, 'recall': 0.2814}}, 'NMF': {'testset_0.30': {'rmse': 2.3114, 'precision': 0.6356, 'recall': 0.3116}}}


In [21]:
print(hybrid_results)

{'SVD': {'precision': 0.6936, 'recall': 0.23879999999999998}, 'NMF': {'precision': 0.6731999999999999, 'recall': 0.25880000000000003}}
